In [ ]:
import MySQLdb as mdb
from datetime import datetime

def connectToSql():
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = 'password'
    db_name = 'securities_master'
    return mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)

In [ ]:
con = connectToSql()
cur = con.cursor()

cur.execute(
    """
    CREATE TABLE exchange (
    id int NOT NULL AUTO_INCREMENT,
    abbrev varchar(32) NOT NULL,
    name varchar(255) NOT NULL,
    city varchar(255) NULL,
    country varchar(255) NULL,
    currency varchar(64) NULL,
    timezone_offset time NULL,
    created_date datetime NOT NULL,
    last_updated_date datetime NOT NULL,

    PRIMARY KEY (id)
    ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;"""
)

cur.execute(
    """
    CREATE TABLE data_vendor (
    id int NOT NULL AUTO_INCREMENT,
    name varchar(64) NOT NULL,
    website_url varchar(255) NULL,
    support_email varchar(255) NULL,
    created_date datetime NOT NULL,
    last_updated_date datetime NOT NULL,
    PRIMARY KEY (id)
    ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;"""
)

cur.execute(
    """CREATE TABLE symbol (
    id int NOT NULL AUTO_INCREMENT,
    exchange_id int NULL,
    ticker varchar(32) NOT NULL,
    instrument varchar(64) NOT NULL,
    name varchar(255) NULL,
    sector varchar(255) NULL,
    currency varchar(32) NULL,
    created_date datetime NOT NULL,
    last_updated_date datetime NOT NULL,
    PRIMARY KEY (id),
    KEY index_exchange_id (exchange_id)
    ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;
    """
)

cur.execute(
    """CREATE TABLE daily_price (
    id int NOT NULL AUTO_INCREMENT,
    data_vendor_id int NOT NULL,
    symbol_id int NOT NULL,
    price_date datetime NOT NULL,
    created_date datetime NOT NULL,
    last_updated_date datetime NOT NULL,
    open_price decimal(19,4) NULL,
    high_price decimal(19,4) NULL,
    low_price decimal(19,4) NULL,
    close_price decimal(19,4) NULL,
    adj_close_price decimal(19,4) NULL,
    volume bigint NULL,
    PRIMARY KEY (id),
    KEY index_data_vendor_id (data_vendor_id),
    KEY index_symbol_id (symbol_id)
    ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;"""
)

cur.close()

In [ ]:
from yahoo_finance import Share

def getSymbols(tickers, instrument):
    symbols = []
    for ticker in tickers:
        tick = Share(ticker)
        symbols.append(
                (
                    ticker,
                    instrument,
                    tick.get_name(),
                    'unknown',
                    tick.get_currency(),
                    datetime.now(), datetime.now()
                )
        )
    return symbols       

In [ ]:
symbols = getSymbols(['CE8.PA', 'CU2.PA', 'ESE.PA', 'STP.PA', 'ANX.PA', 'AUT.PA', 'CD9.PA',
                              'CH5.PA', 'CS5.PA', 'C6E.PA', 'FOO.PA', 'STK.PA', 'S6EW.PA', 'SMC.PA',
                              'STZ.PA', 'MMS.PA', 'FIN.PA'],
                              'etf')
#seems I need to do a new query for this to work (different exchange ?): 
for symbol in getSymbols(['CESL.AS'], 'etf'):
    symbols.append(symbol)
    
for symbol in getSymbols(['SGO.PA', 'ENGI.PA'], 'stock'):
    symbols.append(symbol)
    

In [ ]:
def insertIntodb(db, symbols):
    # Create the insert strings
    column_str = """ticker, instrument, name, sector,
    currency, created_date, last_updated_date
    """
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % (column_str, insert_str)
    
    with db:
        cur = db.cursor()
        cur.executemany(final_str, symbols)

        #ToDo: insertIfNotExist        
        
insertIntodb(con, symbols)

In [ ]:
# insert yahoo data vendor

column_str = """name, website_url, support_email, created_date, last_updated_date"""
insert_str = ("%s, " * 5)[:-2]
final_str = "INSERT INTO data_vendor (%s) VALUES (%s)" % (column_str, insert_str)

with con:
    cur = con.cursor()
    cur.execute(final_str, ('yahoo', '', '', datetime.now(), datetime.now()))

    #ToDo: insertIfNotExist        
